# Scraping LinkedIN Profile Data into a JSON file

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [ ]:
element = input("Paste the body element of the LinkedIN profile")

In [ ]:
soup = BeautifulSoup(element, 'html.parser')

In [ ]:
# Extracting the HTML of the complete introduction box
# that contains the name, company name, and the location
intro = soup.find('div', {'class': 'mt2 relative'})

In [ ]:
name_loc = intro.find("h1")

# Extracting the Name
name = name_loc.get_text().strip()
# strip() is used to remove any extra blank spaces

works_at_loc = intro.find("div", {'class': 'text-body-medium'})
# this gives us the HTML of the tag in which the Company Name is present
# Extracting the Company Name
works_at = works_at_loc.get_text().strip()

location_loc = intro.find_all("span", {'class': "text-body-small inline t-black--light break-words"})
# Extracting the Location
# The 2nd element in the location_loc variable has the location
location = location_loc[0].get_text().strip()

#print("Name -->", name, "\nWorks At -->", works_at, "\nLocation -->", location)

In [ ]:
#Getting the experience details

# Getting the HTML of the Experience section in the profile
experience = soup.find('section', {'id': 'experience-section'}).find('ul')

jobs = experience.find_all('li', {'class': 'pv-entity__position-group-pager pv-profile-section__list-item ember-view'})

job_details={}
role_details={}

count = 0
for job in jobs :
    if(job.find('ul') is None): #If the person had more than one roles in one company-:
        title = job.find('h3', {'class': 't-16 t-black t-bold'}).get_text()
        company = job.find('p', {'class': 'pv-entity__secondary-title t-14 t-black t-normal'}).get_text()
        time = job.find("h4", {'class':'pv-entity__date-range t-14 t-black--light t-normal'}).find('span', {'class': None}).get_text()
        duration = job.find("h4", {'class':'t-14 t-black--light t-normal'}).find('span', {'class': 'pv-entity__bullet-item-v2'}).get_text()
        location = job.find("h4", {'class':'pv-entity__location t-14 t-black--light t-normal block'}).find('span', {'class': None}).get_text()
        desc_temp = job.find("div", {'class':'inline-show-more-text inline-show-more-text--is-collapsed pv-entity__description t-14 t-black t-normal'})
        description = 'NA' if type(desc_temp) is None else desc_temp.get_text()
        job_dict = {
                'company': company,
                'title': title, 
                'time': time,
                'duration': duration,
                'location': location,
                'description': description
               }
    else:
        company = job.find('h3', {'class': 't-16 t-black t-bold'}).find('span', {'class': None}).get_text()
        all_roles = job.find_all('li')
        rcount = 0
        
        for role in all_roles:
            title = role.find("h3", {'class':'t-14 t-black t-bold'}).find('span', {'class': None}).get_text()
            time = role.find("h4", {'class':'pv-entity__date-range t-14 t-black--light t-normal'}).find('span', {'class': None}).get_text()
            duration = role.find("h4", {'class':'t-14 t-black--light t-normal'}).find('span', {'class': 'pv-entity__bullet-item-v2'}).get_text()
            location = role.find("h4", {'class':'pv-entity__location t-14 t-black--light t-normal block'}).find('span', {'class': None}).get_text()
            desc_temp = role.find("div", {'class':'inline-show-more-text inline-show-more-text--is-collapsed pv-entity__description t-14 t-black t-normal'})
            description = 'NA' if type(desc_temp) is None else desc_temp.get_text()
            role_dict = {
                            'title':title,
                            'time':time,
                            'duration':duration,
                            'location': location,
                            'description':description
                          }
            rcount = rcount+1
            role_details.update({'role'+str(rcount): role_dict})
        job_dict = {'company': company, 'roles': role_details}
    
    count = count + 1
    
    job_details.update({'job'+str(count): job_dict})
    
    if(count == 5) :
        break
        

In [ ]:
#Creating a JSON file and transferring all the details in it
import json

my_profile = {
    "name" : name,
    "works_at" : works_at,
    "location" : location,
    "experience" : job_details
}
print(my_profile)
with open("linkedin_profile", "w") as outfile:
    json.dump(my_profile, outfile)